In [1]:
import numpy as np
import math
from tqdm.notebook import tqdm
from functools import cache
import copy

import heapq



In [2]:
with open("input_day_17.txt") as f:
    text = f.read()


test = False
if test:
    text = r"""
2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533
"""

test2 = False
if test2:
    text = """
241
391
"""

test3 = False
if test3:
    text = """
21111
99999
"""

test4 = False
if test4:
    text = """
21111
"""

test5 = False
if test5:
    text = """
111111111111
999999999991
999999999991
999999999991
999999999991
"""

text = text.strip()


In [3]:
G_strs = text.split("\n")
R = len(G_strs)
C = len(G_strs[0])

G = []
for r_str in G_strs:
    G.append([])
    for c in range(len(r_str)):
        G[-1].append(r_str[c])

print(R, C)


141 141


In [4]:
# state is pos, accumulated heat loss, dir, num_straight_steps

def estimated_total_heat_loss(state): 
    current_pos, acc_heat_loss, dir, num_straight_steps = state
    return acc_heat_loss + (R - current_pos[0]) + (C - current_pos[1])

    # for some reason, this takes way way longer
    # return acc_heat_loss + 5 * (R - current_pos[0]) + 5 * (C - current_pos[1])


In [5]:
part = 1

for part in [1, 2]:
    print(f"part: {part}")

    num_states_considered = 0
    best_final_heat_loss = 9999999999999

    min_curr_acc_heat_losses = {}

    q = []
    heapq.heapify(q)

    first_state = ((0, 0), 0, (0, 1), 0)

    heapq.heappush(q, (estimated_total_heat_loss(first_state), first_state))

    while len(q) > 0:

        est, current_state = heapq.heappop(q)

        curr_pos, curr_acc_heat_loss, curr_dir, curr_num_straight_steps = current_state

        key = (curr_pos, curr_dir, curr_num_straight_steps)
        if key in min_curr_acc_heat_losses:
            if min_curr_acc_heat_losses[key] <= curr_acc_heat_loss:
                continue
            else:
                min_curr_acc_heat_losses[key] = curr_acc_heat_loss
        else:
            min_curr_acc_heat_losses[key] = curr_acc_heat_loss
        
        num_states_considered += 1

        if num_states_considered % 100000 == 0:
            print(f"    {num_states_considered} states considered, best final heat loss: {best_final_heat_loss}")

        if curr_acc_heat_loss > best_final_heat_loss:
            continue

        if current_state[0] == (R-1, C-1):
            if not (part == 2 and curr_num_straight_steps < 4):
                best_final_heat_loss = min(best_final_heat_loss, current_state[1])
            continue

        for next_dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        
            if next_dir == (-1, 0) and curr_dir == (1, 0):
                continue
            if next_dir == (1, 0) and curr_dir == (-1, 0):
                continue
            if next_dir == (0, -1) and curr_dir == (0, 1):
                continue
            if next_dir == (0, 1) and curr_dir == (0, -1):
                continue

            if part == 2 and curr_num_straight_steps < 4 and next_dir != curr_dir: # too early to turn
                continue

            if part == 2 and next_dir != curr_dir: # turning, have to take four steps
                if not ((0 <= curr_pos[0] + 4 * next_dir[0] < R) and (0 <= curr_pos[1] + 4 * next_dir[1] < C)):
                    continue
                next_heat_loss = curr_acc_heat_loss
                for n in range(1, 5):
                    next_pos = (curr_pos[0] + n * next_dir[0], curr_pos[1] + n * next_dir[1])
                    next_heat_loss += int(G[next_pos[0]][next_pos[1]])
                
                next_num_straight_steps = 4
            
            else: # just take one step
                next_pos = (curr_pos[0] + next_dir[0], curr_pos[1] + next_dir[1])
            
                if not ((0 <= next_pos[0] < R) and (0 <= next_pos[1] < C)):
                    continue
            
                next_heat_loss = curr_acc_heat_loss + int(G[next_pos[0]][next_pos[1]])
                next_num_straight_steps = 1

            if curr_dir == next_dir:
                next_num_straight_steps = curr_num_straight_steps + 1
            
            if part == 1 and next_num_straight_steps > 3:
                continue
            if part == 2 and next_num_straight_steps > 10:
                continue
            next_state = (next_pos, next_heat_loss, next_dir, next_num_straight_steps)
            heapq.heappush(q, (estimated_total_heat_loss(next_state), next_state))

    print(f"    all states exhausted, best_final_heat_loss = {best_final_heat_loss}")

part: 1
    100000 states considered, best final heat loss: 9999999999999
    200000 states considered, best final heat loss: 742
    all states exhausted, best_final_heat_loss = 742
part: 2
    100000 states considered, best final heat loss: 9999999999999
    200000 states considered, best final heat loss: 9999999999999
    300000 states considered, best final heat loss: 9999999999999
    400000 states considered, best final heat loss: 9999999999999
    500000 states considered, best final heat loss: 918
    all states exhausted, best_final_heat_loss = 918


In [6]:
# 888 is too high
# 778 too high
# 723 too low
# 765 not right
# 748 not right
# 742 right

# part 2:
# 980 too high
# 948 too high
# 942 too high
# 918 right
